## Import all required libraries


In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from tensorflow import keras
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from keras.callbacks import EarlyStopping


## Load Train and Test data

In [59]:
fashion_mnist = keras.datasets.fashion_mnist
(train_full_X, train_full_y), (test_X, test_y) = fashion_mnist.load_data()

In [60]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", 
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

## Designing Fully Connected Neural Network Architecture

Define the model

In [61]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=0.001, input_shape=[28, 28]):   
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu"))
    model.add(Dense(10, activation='softmax'))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", metrics=['accuracy'], optimizer=optimizer)
    return model

In [62]:
keras_reg = keras.wrappers.scikit_learn.KerasClassifier(build_model)

C:\Users\sebas\AppData\Local\Temp\ipykernel_30640\864541393.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasClassifier(build_model)


Run the model

In [63]:
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": [0, 10, 20, 50, 100],
    "learning_rate": [0.0001, 0.001, 0.01],
}

train_X, val_X, train_y, val_y = train_test_split(train_full_X, train_full_y, test_size=0.10, random_state=42)


rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(train_X, train_y, epochs=100,
                  validation_data=(val_X, val_y),
                  callbacks=[EarlyStopping(patience=5)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100
1125/1125 [==============================] - 2s 998us/step - loss: 2.8423 - accuracy: 0.1186 - val_loss: 2.2911 - val_accuracy: 0.1243
Epoch 2/100
1125/1125 [==============================] - 1s 863us/step - loss: 2.2782 - accuracy: 0.1227 - val_loss: 2.2671 - val_accuracy: 0.1363
Epoch 3/100
1125/1125 [==============================] - 1s 853us/step - loss: 2.2589 - accuracy: 0.1299 - val_loss: 2.2573 - val_accuracy: 0.1382
Epoch 4/100
1125/1125 [==============================] - 1s 872us/step - loss: 2.2449 - accuracy: 0.1345 - val_loss: 2.2495 - val_accuracy: 0.1395
Epoch 5/100
1125/1125 [==============================] - 1s 854us/step - loss: 2.2315 - accuracy: 0.1418 - val_loss: 2.2401 - val_accuracy: 0.1398
Epoch 6/100
1125/1125 [==============================] - 1s 856us/step - loss: 2.2185 - accuracy: 0.1498 - val_loss: 2.2295 - val_accuracy: 0.1472
Epoch 7/100
1125/1125 [==============================] - 

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x0000016832229430>,
                   param_distributions={'learning_rate': [0.0001, 0.001, 0.01],
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': [0, 10, 20, 50, 100]},
                   verbose=2)

In [64]:
rnd_search_cv.best_params_

{'n_neurons': 100, 'n_hidden': 2, 'learning_rate': 0.001}

In [65]:
rnd_search_cv.best_score_

0.8243518471717834

In [66]:
rnd_search_cv.best_estimator_

In [69]:
model.evaluate(test_X, test_y)

313/313 [==============================] - 0s 965us/step - loss: 0.5481 - accuracy: 0.8112


[0.5481187701225281, 0.8112000226974487]